<h4>In this notbook I want to do  following steps for an Advance Price Prediction:</h4>

* [Data overview](#Over)
* [Data prepration](#Prep)
    - [Handling Outliers](#outliers)
    - [Handling Missing Data](#mis)
* [Linear Reggresion](#Lin)
* [polynomial Regression](#Pol)
* [Regularization](#Reg)
    - [Ridge](#Ridge)
    - [Lasso](#Lasso)
    - [ElasticNet](#Elas)



<h3>Import all necessary libraries</h3>

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# **Import the dataset:**

In [ ]:
df=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")

<a id="Over"></a>
# Data overview

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.describe

In [ ]:
df.shape

<a id="Prep"></a>
# Data Preparation

<a id="outliers"></a>
# A- Dealing with Outliers

In [ ]:
df.corr()['SalePrice'].sort_values()

finding outliers

In [ ]:
sns.scatterplot(data=df,x='OverallQual', y='SalePrice')
plt.axhline(y=200000,color='r')
plt.axhline(y=650000,color='r')

In [ ]:
df[(df['OverallQual']>8) & (df['SalePrice']<200000)][['SalePrice', 'OverallQual']]


In [ ]:
df[(df['OverallQual']>8) & (df['SalePrice']>650000)][['SalePrice', 'OverallQual']]

In [ ]:
sns.scatterplot(data=df,x='GrLivArea', y='SalePrice')
plt.axhline(y=200000, color='r')
plt.axvline(x=4000, color='r')

In [ ]:
df[(df['GrLivArea']>4000) & (df['SalePrice']>200000)][['SalePrice', 'GrLivArea']]

In [ ]:
sns.scatterplot(data=df,x='GarageCars', y='SalePrice')
plt.axhline(y=650000, color='r')


In [ ]:
df[df['SalePrice']>650000][['SalePrice', 'GarageCars']]

<b>Remove the outliers:</b>

In [ ]:

index_drop=df[df['SalePrice']>650000 ].index
df=df.drop(index_drop, axis=0)
index_drop1=df[(df['OverallQual']>8) & (df['SalePrice']<200000)].index
df=df.drop(index_drop1, axis=0)

In [ ]:
df.corr()['SalePrice'].sort_values()

<a id="mis"></a>
# B-Dealing with Missing Data

Cleaning the Id columns because it doesnt have important information.


In [ ]:
df=df.drop('Id',axis=1)

<h3>number of missing Data in each columns:</h3>


In [ ]:
df.isnull().sum().sort_values()

<h3>The percent of missing data in any feature:</h3>

In [ ]:
100*(df.isnull().sum()/len(df)).sort_values()

<h3>Make a Function to calculate the percent of missing data in each columns (feature) and then sort it</h3>

In [ ]:
def missing_percent(df):
    nan_percent= 100*(df.isnull().sum()/len(df))
    nan_percent= nan_percent[nan_percent>0].sort_values()
    return nan_percent

In [ ]:
nan_percent= missing_percent(df)
nan_percent

In [ ]:
plt.figure(figsize=(12,6),dpi=200)
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(12,6),dpi=200)
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=90)
#Set 1% threshold:
plt.ylim(0,1)

In [ ]:
#columns with missing Data under 1%:
nan_percent[nan_percent<1]

In [ ]:
100/len(df)

It shows Electrical has got just 1 row missing Data.

<h3>The index of a row with missing Data of Electrical:</h3>

In [ ]:
df[df['Electrical'].isnull()]['Electrical']

<h3>removing the row with missing Data of Electrical:
</h3>

In [ ]:
df=df.dropna(axis=0, subset=['Electrical'])

In [ ]:
nan_percent= missing_percent(df)
plt.figure(figsize=(12,6),dpi=200)
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=90)
#Set 1% threshold:
plt.ylim(0,1)

<h3>filling a missing Data with 0 for integer feature and None for object one:</h3>

In [ ]:
df['MasVnrType']= df['MasVnrType'].fillna('None')
df['MasVnrArea']= df['MasVnrArea'].fillna(0)

In [ ]:
nan_percent= missing_percent(df)
plt.figure(figsize=(12,6),dpi=200)
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
bsm=['BsmtQual','BsmtCond','BsmtFinType1','BsmtExposure','BsmtFinType2']
for i in bsm:
    print(type(i))

In [ ]:
df[bsm]=df[bsm].fillna('None')


In [ ]:
nan_percent= missing_percent(df)
plt.figure(figsize=(12,6),dpi=200)
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
garage=['GarageType','GarageFinish','GarageQual','GarageCond']
df[garage]=df[garage].fillna('None')

<h3>replace the amount of missing Data with mean of the GarageYrBlt:</h3>

In [ ]:
df['GarageYrBlt']=df['GarageYrBlt'].fillna(value=df['GarageYrBlt'].mean())

In [ ]:
nan_percent= missing_percent(df)
plt.figure(figsize=(12,6),dpi=200)
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(8,12))
sns.boxplot(data=df, x='LotFrontage', y='Neighborhood')

In [ ]:
df.groupby('Neighborhood')['LotFrontage'].mean()

In [ ]:
df['LotFrontage']=df.groupby('Neighborhood')['LotFrontage'].transform(lambda val: val.fillna(val.mean()))

In [ ]:
nan_percent= missing_percent(df)
plt.figure(figsize=(12,6),dpi=200)
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
df['FireplaceQu']= df['FireplaceQu'].fillna('None')

In [ ]:
nan_percent= missing_percent(df)
plt.figure(figsize=(12,6),dpi=200)
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
df=df.drop(['Fence','Alley','MiscFeature','PoolQC'],axis=1)

In [ ]:
nan_percent= missing_percent(df)
nan_percent

<b>There is no more missing Data</b>

<h3>Dealing with Categorical Data</h3


<h3>Changing Numerical Columns to Categorical</h3>

In [ ]:
df['MSSubClass']

In [ ]:
df.info()


In [ ]:
df['MSSubClass'].unique()

<h3>Convert to String:</h3>

In [ ]:
df['MSSubClass']= df['MSSubClass'].apply(str)

In [ ]:
df.info()

<h3> Creating Dummy Variables:</h3>


In [ ]:
df.select_dtypes(include='object')

In [ ]:
df_num= df.select_dtypes(exclude='object')
df_obj= df.select_dtypes(include='object')

In [ ]:
df_num.info()

In [ ]:
df_obj.info()

In [ ]:
# Converting:
df_obj= pd.get_dummies(df_obj, drop_first=True)

In [ ]:
df_obj.shape

In [ ]:
Final_df= pd.concat([df_num, df_obj], axis=1)
Final_df.head()

#### Now, the Dataset is Ready for any Machine Learing Model & Analysis

<a id="Lin"></a>
# Linear Regression model

In [ ]:
Final_df.info()

In [ ]:
from scipy.stats import norm
plt.figure(figsize=(12,6))
sns.displot(Final_df['SalePrice'])
plt.figure(figsize=(12,6))
sns.displot(Final_df['SalePrice'],color='purple',bins=50)
plt.axvline(x=(Final_df['SalePrice'].mean()), color='r')

<h3>We need to make this normal distributed.</h3>

In [ ]:
Final_df['SalePrice'] = np.log1p(Final_df['SalePrice'])

In [ ]:
from scipy.stats import norm
plt.figure(figsize=(12,6))
sns.displot(Final_df['SalePrice'])
plt.figure(figsize=(12,6))
sns.displot(Final_df['SalePrice'],color='purple')
plt.axvline(x=(Final_df['SalePrice'].mean()), color='r')

<h3>Determine the Features & Target Variable</h3>

In [ ]:
X=Final_df.drop('SalePrice',axis=1)
y=Final_df['SalePrice']

<h3>Split the Dataset to Train & Test</h3>

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

<h3>Train the Model</h3>


In [ ]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X_train, y_train)
pd.DataFrame(model.coef_ , X.columns ,columns=['coefcient'])

<h3>predicting Test Data</h3>

In [ ]:
y_pred= model.predict(X_test)
pd.DataFrame({'Y_Test': y_test,'Y_Pred':y_pred})[:5]

<h3>Evaluating the Model</h3>


In [ ]:
from sklearn import metrics
MAE_linear=metrics.mean_absolute_error(y_test , y_pred)
MSE_linear=metrics.mean_squared_error(y_test , y_pred)
RMSE_linear=np.sqrt(MSE_linear)
pd.DataFrame([MAE_linear,MSE_linear,RMSE_linear], index=['MAE_linear','MSE_linear','RMSE_linear'],columns=['Quantity'])


<h3>Residuals:</h3>


In [ ]:

test_residual=y_test-y_pred
sns.scatterplot(x=y_test,y=y_pred,color='purple' ,s=200)
plt.ylabel('y_pred')
plt.xlabel('y_test')
plt.title('bias of y')

In [ ]:
sns.scatterplot(x=y_test,y=test_residual,s=200)
plt.axhline(y=0,color='red',ls='--')

<h3> As we see ,the residuals show a clear pattern.so we can say,linear regression is not suitable for this model.</h3>

<a id="Pol"></a>
# Polynomial Regression model

<h3>Determine the Features & Target Variable</h3>

In [ ]:
X=Final_df.drop('SalePrice',axis=1)
y=Final_df['SalePrice']

<h3> Preprocessing</h3>


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_converter=PolynomialFeatures(degree=2, include_bias=False)
poly_features=polynomial_converter.fit(X)
poly_features=polynomial_converter.transform(X)


<h3>Split the Data to Train & Test</h3>

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

<h3> Train the Model</h3>


In [ ]:
from sklearn.linear_model import LinearRegression
polymodel=LinearRegression()
polymodel.fit(X_train, y_train)

<h3> Predicting Test Data</h3>


In [ ]:
y_pred=polymodel.predict(X_test)
pd.DataFrame({'Y_Test': y_test,'Y_Pred':y_pred, 'Residuals':(y_test-y_pred) }).head(5)


<h3> Evaluating the Model</h3>

In [ ]:
from sklearn import metrics
MAE_Poly = metrics.mean_absolute_error(y_test,y_pred)
MSE_Poly = metrics.mean_squared_error(y_test,y_pred)
RMSE_Poly = np.sqrt(MSE_Poly)

pd.DataFrame([MAE_Poly, MSE_Poly, RMSE_Poly], index=['MAE_Poly', 'MSE_Poly', 'RMSE_Poly'], columns=['metrics'])


In [ ]:
del df

In [ ]:
# Adjusting Model Parameters
# Train List of RMSE per degree
train_RMSE_list=[]
#Test List of RMSE per degree
test_RMSE_list=[]

for d in range(1,3):
    
    #Preprocessing
    #create poly data set for degree (d)
    polynomial_converter= PolynomialFeatures(degree=d, include_bias=False)
    poly_features= polynomial_converter.fit(X)
    poly_features= polynomial_converter.transform(X)
    
    #Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)
    
    #Train the Model
    polymodel=LinearRegression()
    polymodel.fit(X_train, y_train)
    
    #Predicting on both Train & Test Data
    y_train_pred=polymodel.predict(X_train)
    y_test_pred=polymodel.predict(X_test)
    
    #Evaluating the Model
    
    #RMSE of Train set
    train_RMSE=np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
    
    #RMSE of Test Set
    test_RMSE=np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
      #Append the RMSE to the Train and Test List
    
    train_RMSE_list.append(train_RMSE)
    test_RMSE_list.append(test_RMSE)

In [ ]:
pd.DataFrame({'train_RMSE_list': train_RMSE_list,'test_RMSE_list':test_RMSE_list})

In [ ]:
#**Plot the Polynomial degree VS RMSE**

plt.plot(range(1,3), train_RMSE_list[:13], label='Train RMSE')
plt.plot(range(1,3), test_RMSE_list[:13], label='Test RMSE')

plt.xlabel('Polynomial Degree')
plt.ylabel('RMSE')
plt.legend()

<h3>It shows polynomial can't help to redious errores</h3>

<a id="Reg"></a>
# Regularization:


<h4>Preprocessing</h4>

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_converter= PolynomialFeatures(degree=1, include_bias=False)
poly_features= polynomial_converter.fit_transform(X)

<h4>Split the Data to Train & Test</h4>

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

<h4>Scaling the Data</h4>

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaler.fit(X_train)
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

<a id="Ridge"></a>
# Ridge 


<h4>Train the Model</h4>

In [ ]:
from sklearn.linear_model import Ridge
ridge_model= Ridge(alpha=10)
ridge_model.fit(X_train, y_train)

<h4>predict Test Data</h4>

In [ ]:
y_pred= ridge_model.predict(X_test)

<h4>Evaluating the Model</h4>

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

MAE_Ridge= mean_absolute_error(y_test, y_pred)
MSE_Ridge= mean_squared_error(y_test, y_pred)
RMSE_Ridge= np.sqrt(MSE_Ridge)
pd.DataFrame([MAE_Ridge, MSE_Ridge, RMSE_Ridge], index=['MAE_Ridge', 'MSE_Ridge', 'RMSE_Ridge'], columns=['metrics'])

<h4>Coosing an alpha value with Cross-Validation</h4>

<h4>Train the Model</h4>

In [ ]:
from sklearn.linear_model import RidgeCV
ridge_cv_model=RidgeCV(alphas=(0.1, 1.0, 10.0), scoring='neg_mean_absolute_error')
ridge_cv_model.fit(X_train, y_train)

In [ ]:
ridge_cv_model.alpha_

<h4>Predicting Test Data</h4>

In [ ]:
y_pred_ridge= ridge_cv_model.predict(X_test)

In [ ]:
MAE_ridge= mean_absolute_error(y_test, y_pred_ridge)
MSE_ridge= mean_squared_error(y_test, y_pred_ridge)
RMSE_ridge= np.sqrt(MSE_ridge)
pd.DataFrame([MAE_ridge, MSE_ridge, RMSE_ridge], index=['MAE_ridge_CV', 'MSE_ridge_CV', 'RMSE_ridge_CV'], columns=['Ridge Metrics'])

In [ ]:
ridge_cv_model.coef_

<a id="Lasso"></a>
# Lasso


In [ ]:
from sklearn.linear_model import LassoCV
lasso_cv_model= LassoCV(eps=0.01, n_alphas=100, cv=5)
lasso_cv_model.fit(X_train, y_train)

In [ ]:
lasso_cv_model.alpha_

In [ ]:
y_pred_lasso= lasso_cv_model.predict(X_test)
MAE_Lasso= mean_absolute_error(y_test, y_pred_lasso)
MSE_Lasso= mean_squared_error(y_test, y_pred_lasso)
RMSE_Lasso= np.sqrt(MSE_Lasso)

In [ ]:
pd.DataFrame([MAE_Lasso, MSE_Lasso, RMSE_Lasso], index=['MAE_Lasso', 'MSE_Lasso', 'RMSE_Lasso'], columns=['Lasso Metrics'])

In [ ]:
lasso_cv_model.coef_

<a id="Elas"></a>
# Elastic Net


In [ ]:
from sklearn.linear_model import ElasticNetCV
elastic_model= ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],cv=5, max_iter=100000)
elastic_model.fit(X_train, y_train)

In [ ]:
elastic_model.l1_ratio_

In [ ]:
y_pred_elastic=elastic_model.predict(X_test)

In [ ]:
MAE_Elastic= mean_absolute_error(y_test, y_pred_elastic)
MSE_Elastic= mean_squared_error(y_test, y_pred_elastic)
RMSE_Elastic= np.sqrt(MSE_Elastic)

In [ ]:
pd.DataFrame([MAE_Elastic, MSE_Elastic, RMSE_Elastic], index=['MAE_Elastic', 'MSE_Elastic', 'RMSE_Elastic'], columns=['Elastic Metrics'])

In [ ]:
elastic_model.coef_